In [1]:
import csv
import numpy as np

In [2]:
header = []
labels = []
features = []
with open("titanic_data.csv") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
#         print(row)
        labels.append(int(row[0]))
        features.append([float(x) for x in row[1:]])

In [3]:
header = np.array(header)
labels = np.array(labels)
features = np.array(features)

In [4]:
print("header: ", header.shape, "labels: ", labels.shape, "features: ", features.shape)

header:  (7,) labels:  (887,) features:  (887, 6)


In [5]:
#theta_hat_mse
theta_mse = np.linalg.pinv(features.T @ features) @ features.T @ labels
theta_mse = np.insert(theta_mse, 0, 1.0) #add to construct D+1 vector
theta_mse

array([ 1.        ,  0.02134656,  0.58003744,  0.00212285, -0.03574227,
       -0.02226383,  0.00216599])

In [6]:
#add 1s to keep size the same as theta
column = np.ones(features.shape[0])
features = np.insert(features, 0, column, axis=1)
features

array([[ 1.    ,  3.    ,  0.    , ...,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  1.    ,  1.    , ...,  1.    ,  0.    , 71.2833],
       [ 1.    ,  3.    ,  1.    , ...,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 1.    ,  3.    ,  1.    , ...,  1.    ,  2.    , 23.45  ],
       [ 1.    ,  1.    ,  0.    , ...,  0.    ,  0.    , 30.    ],
       [ 1.    ,  3.    ,  0.    , ...,  0.    ,  0.    ,  7.75  ]])

In [7]:
eta = 0.05
steps = 1000

In [8]:
def log_likelihood(labels, features, theta_hat):
    likelihood_total = 0.0
    for i in range(labels.shape[0]):
        term_neg = -1 * theta_hat.T @ features[i]
        term = theta_hat.T @ features[i]
        try:
#             likelihood_left = labels[i] * np.log(1 / (1 + np.exp(term_neg)))
            log_left = np.log(1 / (1 + np.exp(term_neg)))
        except:
            log_left = np.log(1)
        try:
#             likelihood_right = (1 - labels[i]) * np.log(1 / (1 + np.exp(term)))
            log_right = np.log(1 / (1 + np.exp(term)))
        except:
            log_right = np.log(1)
        likelihood = labels[i] * log_left + (1 - labels[i]) * log_right
        likelihood_total += likelihood
    return likelihood_total

In [9]:
log_likelihood(labels, features, np.array([1, 0, 0, 0, 0, 0, 0]))

-822.863116828655

In [10]:
def gradient(labels, features, theta_hat):
    gradient_total = 0.0
    for i in range(labels.shape[0]):
        inner_term = -1 * theta_hat.T @ features[i]
        try:
            gradient = labels[i] - (1 / (1 + np.exp(inner_term))) * features[i]
        except OverflowError:
#             print("catch error")
            gradient = labels[i] * features[i]
        gradient_total += gradient
    return gradient_total

In [11]:
gradient(labels, features, theta_mse)

array([  -359.94948546,  -1266.18808437,     75.02677852, -20396.54418262,
          -25.44073953,     67.35652564, -23196.38032939])

In [13]:
def predict(sample, theta_hat):
    try:
        test = 1 / (1 + np.exp(-1 * theta_hat.T @ sample))
        return test > 0.5
    except:
        return 0

    #check results
def accuracy_check(theta):
    correct = 0
    for i in range(features.shape[0]):
        prediction = predict(features[i], theta)
        if prediction == labels[i]:
            correct += 1
    accuracy = correct / features.shape[0]
    return accuracy

In [14]:
theta = theta_mse #initialize theta
for t in range(steps):
    likelihood = log_likelihood(labels, features, theta)
#     print(likelihood)
    
    theta = theta + eta * gradient(labels, features, theta)
    print(theta)
    accuracy = accuracy_check(theta)
    print("accuracy ", accuracy)

[  -16.99747427   -63.28805766     4.33137636 -1019.82508628
    -1.30777925     3.34556245 -1159.81685048]
accuracy  0.6144306651634723
[ 1.02525727e-01 -4.61880577e+01  2.14313764e+01 -1.00272509e+03
  1.57922207e+01  2.04455624e+01 -1.14271685e+03]
accuracy  0.6144306651634723
[   17.20252573   -29.08805766    38.53137636  -985.62508628
    32.89222075    37.54556245 -1125.61685048]
accuracy 

C:\Users\caday\AppData\Local\Temp\ipykernel_10800\3890977658.py:3: RuntimeWarning: overflow encountered in exp
  test = 1 / (1 + np.exp(-1 * theta_hat.T @ sample))
C:\Users\caday\AppData\Local\Temp\ipykernel_10800\953314346.py:8: RuntimeWarning: overflow encountered in exp
  log_left = np.log(1 / (1 + np.exp(term_neg)))
C:\Users\caday\AppData\Local\Temp\ipykernel_10800\953314346.py:8: RuntimeWarning: divide by zero encountered in log
  log_left = np.log(1 / (1 + np.exp(term_neg)))
C:\Users\caday\AppData\Local\Temp\ipykernel_10800\953314346.py:16: RuntimeWarning: invalid value encountered in multiply
  likelihood = labels[i] * log_left + (1 - labels[i]) * log_right
C:\Users\caday\AppData\Local\Temp\ipykernel_10800\2657821434.py:6: RuntimeWarning: overflow encountered in exp
  gradient = labels[i] - (1 / (1 + np.exp(inner_term))) * features[i]


 0.6144306651634723
[   34.30252573   -11.98805766    55.63137636  -968.52508628
    49.99222075    54.64556245 -1108.51685048]
accuracy  0.6144306651634723
[   51.40252573     5.11194234    72.73137636  -951.42508628
    67.09222075    71.74556245 -1091.41685048]
accuracy  0.6144306651634723
[   68.50252573    22.21194234    89.83137636  -934.32508628
    84.19222075    88.84556245 -1074.31685048]
accuracy  0.6144306651634723
[   85.60252573    39.31194234   106.93137636  -917.22508628
   101.29222075   105.94556245 -1057.21685048]
accuracy  0.6144306651634723
[  102.70252573    56.41194234   124.03137636  -900.12508628
   118.39222075   123.04556245 -1040.11685048]
accuracy  0.6144306651634723
[  119.80252573    73.51194234   141.13137636  -883.02508628
   135.49222075   140.14556245 -1023.01685048]
accuracy  0.6144306651634723
[  136.90252573    90.61194234   158.23137636  -865.92508628
   152.59222075   157.24556245 -1005.91685048]
accuracy  0.6144306651634723
[ 154.00252573  107.7

C:\Users\caday\AppData\Local\Temp\ipykernel_10800\953314346.py:13: RuntimeWarning: overflow encountered in exp
  log_right = np.log(1 / (1 + np.exp(term)))
C:\Users\caday\AppData\Local\Temp\ipykernel_10800\953314346.py:13: RuntimeWarning: divide by zero encountered in log
  log_right = np.log(1 / (1 + np.exp(term)))


accuracy  0.616685456595265
[ 803.15252573  755.56194234  824.63137636 -202.88808628  818.49222075
  823.29556245 -345.15226548]
accuracy  0.6189402480270575
[ 819.95252573  771.76194234  841.53137636 -187.10908628  835.29222075
  840.04556245 -331.12518048]
accuracy  0.6200676437429538
[ 836.70252573  787.81194234  858.38137636 -171.38008628  852.09222075
  856.69556245 -317.88518048]
accuracy  0.6200676437429538
[ 853.25252573  803.26194234  875.08137636 -156.62608628  868.49222075
  873.14556245 -307.98622048]
accuracy  0.62119503945885
[ 869.55252573  817.96194234  891.63137636 -144.32208628  884.69222075
  889.39556245 -300.91747048]
accuracy  0.6178128523111612
[ 885.60237833  831.91150015  908.03122896 -134.52088686  900.49207335
  905.44556245 -297.47383818]
accuracy  0.6200676437429538
[ 901.25228334  844.66129242  924.18121121 -128.91867379  915.79207335
  921.19554469 -299.6919242 ]
accuracy  0.6178128523111612
[ 916.50164943  856.26003035  940.08121697 -129.41508499  930.79

...

accuracy  0.6144306651634723
[11293.72809793  8773.50274667 11776.05737466 -1373.21145151
 11225.88951687 11638.27004049 -3557.19095198]
accuracy  0.6155580608793687
[11308.97992891  8785.20465878 11792.00562481 -1375.43750358
 11241.08426732 11654.01829064 -3561.10447994]
accuracy  0.616685456595265
[11324.2791113   8796.95302356 11807.90562481 -1375.80067341
 11256.13426732 11669.71829064 -3566.24677992]
accuracy  0.6144306651634723
[11339.47911139  8808.50302383 11823.8056249  -1378.24667314
 11271.18426759 11685.41829073 -3571.38907762]
accuracy  0.6144306651634723
[11354.68800642  8820.07970876 11839.71451977 -1380.66598504
 11286.26095219 11701.12718559 -3576.30485473]
accuracy  0.6155580608793687
[11369.96691195  8831.83672769 11855.66372765 -1382.34071105
 11301.45857585 11716.87639348 -3580.19399225]
accuracy  0.6144306651634723
[11385.16802658  8843.38895696 11871.56372765 -1384.76330374
 11316.50857585 11732.57639348 -3585.33629224]
accuracy  0.616685456595265
[11400.4680120

accuracy  0.6144306651634723
[12208.25847528  9472.45010212 12730.9439754  -1482.7923404
 12131.54931909 12581.15664123 -3843.15015692]
accuracy  0.6144306651634723
[12223.45847578  9484.00010356 12746.84397585 -1485.23833803
 12146.59932043 12596.85664167 -3848.29244557]
accuracy  0.6155580608793687
[12238.70325643  9495.66550044 12762.76981144 -1487.20898511
 12161.72682721 12612.58247727 -3852.77679826]
accuracy  0.616685456595265
[12253.98278316  9507.37459535 12778.66985288 -1487.98566965
 12176.77695153 12628.28251871 -3857.9180429 ]
accuracy  0.6144306651634723
[12269.1963153   9518.96519178 12794.58338503 -1490.39107321
 12191.86754796 12643.99605085 -3862.71572386]
accuracy  0.6155580608793687
[12284.43736873  9530.63833861 12810.52442501 -1492.71367093
 12207.0406679  12659.73709083 -3866.81287104]
accuracy  0.616685456595265
[12299.71549508  9542.34459134 12826.42442503 -1493.51901792
 12222.09066797 12675.43709085 -3871.95517047]
accuracy  0.6144306651634723
[12314.91550627

[13137.96251361 10182.86746392 13701.70326056 -1594.88066846
 13052.17717456 13539.71592638 -4134.94729966]
accuracy  0.6155580608793687
[13153.21117052 10194.56343331 13717.65191611 -1597.18067317
 13067.37314123 13555.46458194 -4138.85050321]
accuracy  0.6144306651634723
[13168.42876742 10206.14862711 13733.55191612 -1599.25713833
 13082.42314123 13571.16458194 -4143.99280318]
accuracy  0.616685456595265
[13183.72632732 10217.89378177 13749.45195098 -1599.65500798
 13097.47324582 13586.8646168  -4149.13421532]
accuracy  0.6144306651634723
[13198.93040047 10229.45600123 13765.35602413 -1602.08878853
 13112.53546528 13602.56868996 -4154.17278558]
accuracy  0.6155580608793687
[13214.18032617 10241.15577822 13781.30594974 -1604.38500992
 13127.7352421  13618.31861556 -4158.04364512]
accuracy  0.6144306651634723
[13229.38153105 10252.70818799 13797.20594975 -1606.80570757
 13142.78524212 13634.01861557 -4163.18594492]
accuracy  0.616685456595265
[13244.68234772 10264.4606677  13813.106796

[14067.74730543 10893.49030675 14672.50651974 -1706.06470553
 13972.93695211 14498.31918557 -4425.62456897]
accuracy  0.6144306651634723
[14082.94730549 10905.04030693 14688.4065198  -1708.51070535
 13987.9869523  14514.01918563 -4430.7668674 ]
accuracy  0.6144306651634723
[14098.15374286 10916.60961898 14704.31295711 -1710.93739213
 14003.05626423 14529.72562294 -4435.74523039]
accuracy  0.6155580608793687
[14113.41912868 10928.34016921 14720.26273569 -1712.90630418
 14018.25559995 14545.47540152 -4439.61983432]
accuracy  0.616685456595265
[14128.69089543 10940.0337027  14736.16273569 -1713.84520255
 14033.30559996 14561.17540152 -4444.7621343 ]
accuracy  0.6144306651634723
[14143.8908962  10951.58370499 14752.06273646 -1716.29120023
 14048.35560225 14576.87540228 -4449.90441481]
accuracy  0.6155580608793687
[14159.12397182 10963.23222366 14767.99510388 -1718.62522589
 14063.50270453 14592.60776971 -4454.22242325]
accuracy  0.616685456595265
[14174.42358425 10974.98145156 14783.895106

[14982.2053329  11592.26814576 15627.36830383 -1816.71824219
 14878.52230436 15441.18096965 -4712.21577227]
accuracy  0.6144306651634723
[14997.40534431 11603.81818    15643.26831523 -1819.16420766
 14893.57233857 15456.88098105 -4717.35778192]
accuracy  0.6155580608793687
[15012.66052573 11615.52676745 15659.21653984 -1821.31944091
 14908.76701241 15472.62920567 -4721.27196015]
accuracy  0.616685456595265
[15027.95871683 11627.27314966 15675.11653984 -1821.70342774
 14923.81701241 15488.32920567 -4726.41426014]
accuracy  0.6144306651634723
[15043.15871691 11638.82314989 15691.01653992 -1824.14942751
 14938.86701264 15504.02920574 -4731.55655819]
accuracy  0.6144306651634723
[15058.36647125 11650.39641278 15706.92429414 -1826.57216241
 14953.9402753  15519.73695996 -4736.50138379]
accuracy  0.6155580608793687
[15073.64019821 11662.14343291 15722.87386034 -1828.36208789
 14969.13897391 15535.48652617 -4740.38139616]
accuracy  0.6144306651634723
[15088.84912003 11673.71127655 15738.77386

In [ ]:
test = [-8.87474273e+01, -4.62091908e+03,  2.08571393e+03, -1.00272719e+05,
  1.58276056e+03,  2.04676036e+03, -1.14271899e+05]

In [ ]:
log_likelihood(labels, features, np.array(test))

In [ ]:
theta